In [1]:
# import h2o package and specific estimator 
import h2o
from h2o.automl import H2OAutoML
import random, os, sys
from datetime import datetime
import pandas as pd
import logging
import csv
import optparse
import time
import json
from distutils.util import strtobool
import psutil

import warnings
warnings.filterwarnings('ignore')

In [2]:
h2o.init(strict_version_check=False) # start h2o

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,9 hours 4 mins
H2O cluster timezone:,America/New_York
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.1
H2O cluster version age:,18 days
H2O cluster name:,H2O_from_python_newzysharma_iv2iht
H2O cluster total nodes:,1
H2O cluster free memory:,1.711 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [3]:
#importing data to the server
hp = h2o.import_file(path="hour.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
#Displaying the head
hp.head()

instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
1,2011-01-01 00:00:00,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0,3,13,16
2,2011-01-01 00:00:00,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0,8,32,40
3,2011-01-01 00:00:00,1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0,5,27,32
4,2011-01-01 00:00:00,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0,3,10,13
5,2011-01-01 00:00:00,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0,0,1,1
6,2011-01-01 00:00:00,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,0.0896,0,1,1
7,2011-01-01 00:00:00,1,0,1,6,0,6,0,1,0.22,0.2727,0.8,0,2,0,2
8,2011-01-01 00:00:00,1,0,1,7,0,6,0,1,0.2,0.2576,0.86,0,1,2,3
9,2011-01-01 00:00:00,1,0,1,8,0,6,0,1,0.24,0.2879,0.75,0,1,7,8
10,2011-01-01 00:00:00,1,0,1,9,0,6,0,1,0.32,0.3485,0.76,0,8,6,14


In [5]:
hp.describe()

Rows:17379
Cols:17




,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
type,int,time,int,int,int,int,int,int,int,int,real,real,real,real,int,int,int
mins,1.0,1293840000000.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.02,0.0,0.0,0.0,0.0,0.0,1.0
mean,8690.0,1325477314552.0461,2.501639910236492,0.5025605615973301,6.5377754761493785,11.546751826917548,0.028770355026181024,3.003682605443351,0.6827205247712756,1.425283387997008,0.4969871684216584,0.47577510213476026,0.6272288394038784,0.1900976063064618,35.67621842453536,153.78686920996606,189.4630876345014
maxs,17379.0,1356912000000.0,4.0,1.0,12.0,23.0,1.0,6.0,1.0,4.0,1.0,1.0,1.0,0.8507,367.0,886.0,977.0
sigma,5017.029499614288,18150225217.779854,1.1069181394480765,0.5000078290910197,3.438775713750168,6.914405095264493,0.16716527638437123,2.005771456110988,0.4654306335238829,0.6393568777542534,0.19255612124972193,0.17185021563535932,0.19292983406291514,0.1223402285727905,49.30503038705309,151.35728591258314,181.38759909186476
zeros,0,0,0,8645,0,726,16879,2502,5514,0,0,2,22,2180,1581,24,0
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,1.0,2011-01-01 00:00:00,1.0,0.0,1.0,0.0,0.0,6.0,0.0,1.0,0.24,0.2879,0.81,0.0,3.0,13.0,16.0
1,2.0,2011-01-01 00:00:00,1.0,0.0,1.0,1.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.8,0.0,8.0,32.0,40.0
2,3.0,2011-01-01 00:00:00,1.0,0.0,1.0,2.0,0.0,6.0,0.0,1.0,0.22,0.2727,0.8,0.0,5.0,27.0,32.0


In [6]:
# Functions

def alphabet(n):
  alpha='0123456789abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'    
  str=''
  r=len(alpha)-1   
  while len(str)<n:
    i=random.randint(0,r)
    str+=alpha[i]   
  return str
  
  
def set_meta_data(analysis,run_id,server,data,test,model_path,target,run_time,classification,scale,model,balance,balance_threshold,name,path,nthreads,min_mem_size):
  m_data={}
  m_data['start_time'] = time.time()
  m_data['target']=target
  m_data['server_path']=server
  m_data['data_path']=data 
  m_data['test_path']=test
  m_data['max_models']=model
  m_data['run_time']=run_time
  m_data['run_id'] =run_id
  m_data['scale']=scale
  m_data['classification']=classification
  m_data['scale']=False
  m_data['model_path']=model_path
  m_data['balance']=balance
  m_data['balance_threshold']=balance_threshold
  m_data['project'] =name
  m_data['end_time'] = time.time()
  m_data['execution_time'] = 0.0
  m_data['run_path'] =path
  m_data['nthreads'] = nthreads
  m_data['min_mem_size'] = min_mem_size
  m_data['analysis'] = analysis
  return m_data


def dict_to_json(dct,n):
  j = json.dumps(dct, indent=4)
  f = open(n, 'w')
  print(j, file=f)
  f.close()
  
  
def stackedensemble(mod):
    coef_norm=None
    try:
      metalearner = h2o.get_model(mod.metalearner()['name'])
      coef_norm=metalearner.coef_norm()
    except:
      pass        
    return coef_norm

def stackedensemble_df(df):
    bm_algo={ 'GBM': None,'GLM': None,'DRF': None,'XRT': None,'Dee': None}
    for index, row in df.iterrows():
      if len(row['model_id'])>3:
        key=row['model_id'][0:3]
        if key in bm_algo:
          if bm_algo[key] is None:
                bm_algo[key]=row['model_id']
    bm=list(bm_algo.values()) 
    bm=list(filter(None.__ne__, bm))             
    return bm

def se_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['auc']=modl.auc()   
    d['roc']=modl.roc()
    d['mse']=modl.mse()   
    d['null_degrees_of_freedom']=modl.null_degrees_of_freedom()
    d['null_deviance']=modl.null_deviance()
    d['residual_degrees_of_freedom']=modl.residual_degrees_of_freedom()   
    d['residual_deviance']=modl.residual_deviance()
    d['rmse']=modl.rmse()
    return d

def get_model_by_algo(algo,models_dict):
    mod=None
    mod_id=None    
    for m in list(models_dict.keys()):
        if m[0:3]==algo:
            mod_id=m
            mod=h2o.get_model(m)      
    return mod,mod_id     
    
    
def gbm_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    return d
    
    
def dl_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    return d
    
    
def drf_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    d['roc']=modl.roc()      
    return d
    
def xrt_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['varimp']=modl.varimp()  
    d['roc']=modl.roc()      
    return d
    
    
def glm_stats(modl):
    d={}
    d['algo']=modl.algo
    d['model_id']=modl.model_id   
    d['coef']=modl.coef()  
    d['coef_norm']=modl.coef_norm()      
    return d
    
def model_performance_stats(perf):
    d={}
    try:    
      d['mse']=perf.mse()
    except:
      pass      
    try:    
      d['rmse']=perf.rmse() 
    except:
      pass      
    try:    
      d['null_degrees_of_freedom']=perf.null_degrees_of_freedom()
    except:
      pass      
    try:    
      d['residual_degrees_of_freedom']=perf.residual_degrees_of_freedom()
    except:
      pass      
    try:    
      d['residual_deviance']=perf.residual_deviance() 
    except:
      pass      
    try:    
      d['null_deviance']=perf.null_deviance() 
    except:
      pass      
    try:    
      d['aic']=perf.aic() 
    except:
      pass      
    try:
      d['logloss']=perf.logloss() 
    except:
      pass    
    try:
      d['auc']=perf.auc()
    except:
      pass  
    try:
      d['gini']=perf.gini()
    except:
      pass    
    return d
    
def impute_missing_values(df, x, scal=False):
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in x:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    _ = df[reals].impute(method='mean')
    _ = df[ints].impute(method='median')
    if scal:
        df[reals] = df[reals].scale()
        df[ints] = df[ints].scale()    
    return


def get_independent_variables(df, targ):
    C = [name for name in df.columns if name != targ]
    # determine column types
    ints, reals, enums = [], [], []
    for key, val in df.types.items():
        if key in C:
            if val == 'enum':
                enums.append(key)
            elif val == 'int':
                ints.append(key)            
            else: 
                reals.append(key)    
    x=ints+enums+reals
    return x
    
def get_all_variables_csv(i):
    ivd={}
    try:
      iv = pd.read_csv(i,header=None)
    except:
      sys.exit(1)    
    col=iv.values.tolist()[0]
    dt=iv.values.tolist()[1]
    i=0
    for c in col:
      ivd[c.strip()]=dt[i].strip()
      i+=1        
    return ivd
    
    

def check_all_variables(df,dct,y=None):     
    targ=list(dct.keys())     
    for key, val in df.types.items():
        if key in targ:
          if dct[key] not in ['real','int','enum']:                      
            targ.remove(key)  
    for key, val in df.types.items():
        if key in targ:            
          if dct[key] != val:
            print('convert ',key,' ',dct[key],' ',val)
            if dct[key]=='enum':
                try:
                  df[key] = df[key].asfactor() 
                except:
                  targ.remove(key)                 
            if dct[key]=='int': 
                try:                
                  df[key] = df[key].asnumeric() 
                except:
                  targ.remove(key)                  
            if dct[key]=='real':
                try:                
                  df[key] = df[key].asnumeric()  
                except:
                  targ.remove(key)                  
    if y is None:
      y=df.columns[-1] 
    if y in targ:
      targ.remove(y)
    else:
      y=targ.pop()            
    return targ    
    
def predictions(mod,data,run_id):
    test = h2o.import_file(data)
    mod_perf=mod_best.model_performance(test)
              
    stats_test={}
    stats_test=model_performance_stats(mod_perf)

    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 

    try:    
      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf[0].table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass

    predictions = mod_best.predict(test)
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return

def predictions_test(mod,test,run_id):
    mod_perf=mod_best.model_performance(test)          
    stats_test={}
    stats_test=model_performance_stats(mod_perf)
    n=run_id+'_test_stats.json'
    dict_to_json(stats_test,n) 
    try:
      cf=mod_perf.confusion_matrix()
#      cf=mod_perf.confusion_matrix(metrics=["f1","f2","f0point5","accuracy","precision","recall","specificity","absolute_mcc","min_per_class_accuracy","mean_per_class_accuracy"])
      cf_df=cf.table.as_data_frame()
      cf_df.to_csv(run_id+'_test_confusion_matrix.csv')
    except:
      pass
    predictions = mod_best.predict(test)    
    predictions_df=test.cbind(predictions).as_data_frame() 
    predictions_df.to_csv(run_id+'_predictions.csv')
    return predictions

def check_X(x,df):
    for name in x:
        if name not in df.columns:
          x.remove(name)  
    return x    
    
    
def get_stacked_ensemble(lst):
    se=None
    for model in model_set:
      if 'BestOfFamily' in model:
        se=model
    if se is None:     
      for model in model_set:
        if 'AllModels'in model:
          se=model           
    return se       
    
def get_variables_types(df):
    d={}
    for key, val in df.types.items():
        d[key]=val           
    return d    
    
#  End Functions

In [7]:
all_variables=None

## Model with runtime 1350 seconds


In [8]:
# Assume the following are passed by the user from the web interface

'''
Need a user id and project id?

'''
target='cnt' 
data_file='hour.csv'
run_time=1350
run_id='1350' # Just some arbitrary ID
server_path='//Users/newzysharma/Desktop/Desktop/info6105/INFO6105-FinalProject/HyperparametersDB/1350'
classification=False
scale=False
max_models=None
balance_y=False # balance_classes=balance_y
balance_threshold=0.2
project ="HyperparameterDB_Project"  # project_name = project

In [9]:
# Use local data file or download from some type of bucket
import os

data_path=os.path.join(server_path,data_file)
data_path

'//Users/newzysharma/Desktop/Desktop/info6105/INFO6105-FinalProject/HyperparametersDB/1350/hour.csv'

In [10]:
# assign target and inputs for logistic regression
y = target
X = [name for name in hp.columns if name != y]
print(y)
print(X)

cnt
['instant', 'dteday', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'casual', 'registered']


In [11]:
# determine column types
ints, reals, enums = [], [], []
for key, val in hp.types.items():
    if key in X:
        if val == 'enum':
            enums.append(key)
        elif val == 'int':
            ints.append(key)            
        else: 
            reals.append(key)

print(ints)
print(enums)
print(reals)

['instant', 'season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'casual', 'registered']
[]
['dteday', 'temp', 'atemp', 'hum', 'windspeed']


In [12]:
# impute missing values
_ = hp[reals].impute(method='mean')
_ = hp[ints].impute(method='median')

if scale:
    hp[reals] = hp[reals].scale()
    hp[ints] = hp[ints].scale()

In [13]:
if classification:
    class_percentage = y_balance=df[y].mean()[0]/(df[y].max()-df[y].min())
    if class_percentage < balance_threshold:
        balance_y=True
        

print(run_time)
type(run_time)

1350


int

## Cross-validate rather than take a test training split with 1350 seconds

In [14]:
# automl
# runs for run_time seconds then builds a stacked ensemble

aml = H2OAutoML(max_runtime_secs=run_time,project_name = project) # init automl, run for 1350 seconds
aml.train(x=X,  
           y=y,
           training_frame=hp)

AutoML progress: |████████████████████████████████████████████████████████| 100%


## Leaderboard

In [15]:
# view leaderboard
lb = aml.leaderboard
lb

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_3_AutoML_20190419_001144,11.6071,3.40693,11.6071,1.86843,0.0258769
GBM_1_AutoML_20190419_001144,11.928,3.45369,11.928,2.11871,0.0454763
GBM_2_AutoML_20190419_015759,13.1177,3.62184,13.1177,2.26602,0.051377
DeepLearning_1_AutoML_20190419_001144,13.4984,3.67402,13.4984,2.46396,nan
GBM_1_AutoML_20190419_015759,14.8593,3.85478,14.8593,2.37037,nan
GBM_4_AutoML_20190419_001144,17.3214,4.1619,17.3214,2.4208,nan
XGBoost_1_AutoML_20190418_233434,17.4453,4.17676,17.4453,2.44703,0.0424657
XGBoost_1_AutoML_20190419_001144,18.109,4.25547,18.109,2.43733,0.0445539
XGBoost_2_AutoML_20190418_233434,19.0881,4.36899,19.0881,2.48959,0.0526343
XGBoost_3_AutoML_20190418_165441,19.6166,4.42906,19.6166,2.86906,0.0793301


In [16]:
aml.leader

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_3_AutoML_20190419_001144


ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 2.039361059245714
RMSE: 1.4280619941885275
MAE: 0.9585966193344686
RMSLE: 0.01821836299931219
Mean Residual Deviance: 2.039361059245714

ModelMetricsRegression: gbm
** Reported on cross-validation data. **

MSE: 11.607149260444618
RMSE: 3.4069266590938847
MAE: 1.868428412147364
RMSLE: 0.025876912011592763
Mean Residual Deviance: 11.607149260444618
Cross-Validation Metrics Summary: 


,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
mae,1.8684285,0.0082047,1.8704685,1.8793072,1.8466274,1.8771082,1.8686309
mean_residual_deviance,11.60713,0.6483614,10.321971,11.190566,12.924045,12.33351,11.265555
mse,11.60713,0.6483614,10.321971,11.190566,12.924045,12.33351,11.265555
r2,0.9996471,0.0000203,0.9996867,0.9996625,0.9996049,0.9996259,0.9996557
residual_deviance,11.60713,0.6483614,10.321971,11.190566,12.924045,12.33351,11.265555
rmse,3.404269,0.0950832,3.2127824,3.3452303,3.5950027,3.51191,3.35642
rmsle,0.0258275,0.0011303,0.0254128,0.0283115,0.0233743,0.0256065,0.0264324


Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance
,2019-04-19 00:27:35,15.414 sec,0.0,181.3823804,142.3998489,32899.5679309
,2019-04-19 00:27:36,15.480 sec,5.0,107.5852801,84.3992630,11574.5924958
,2019-04-19 00:27:36,15.544 sec,10.0,64.1991553,50.2256912,4121.5315458
,2019-04-19 00:27:36,15.613 sec,15.0,38.2374594,29.8367780,1462.1033029
,2019-04-19 00:27:36,15.677 sec,20.0,23.5604922,18.2173502,555.0967911
---,---,---,---,---,---,---
,2019-04-19 00:27:38,17.663 sec,205.0,1.4749113,0.9846722,2.1753632
,2019-04-19 00:27:38,17.712 sec,210.0,1.4636709,0.9781465,2.1423324
,2019-04-19 00:27:38,17.762 sec,215.0,1.4474807,0.9692166,2.0952004
,2019-04-19 00:27:38,17.811 sec,220.0,1.4375300,0.9638242,2.0664926



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
registered,2582412032.0000000,1.0,0.9230252
casual,126951304.0000000,0.0491600,0.0453759
hr,45026460.0000000,0.0174358,0.0160937
weekday,9260662.0,0.0035861,0.0033100
workingday,8981101.0,0.0034778,0.0032101
temp,8863775.0,0.0034324,0.0031682
dteday,8359622.5,0.0032371,0.0029880
atemp,3492493.7500000,0.0013524,0.0012483
instant,2108263.0,0.0008164,0.0007536
hum,979629.25,0.0003793,0.0003501


In [17]:
aml.leader.algo

'gbm'

## Ensemble Exploration

In [18]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
aml_leaderboard_df

,model_id,mean_residual_deviance,rmse,mse,mae,rmsle
0,GBM_3_AutoML_20190419_001144,11.607149,3.406927,11.607149,1.868428,0.025877
1,GBM_1_AutoML_20190419_001144,11.927990,3.453692,11.927990,2.118712,0.045476
2,GBM_2_AutoML_20190419_015759,13.117716,3.621839,13.117716,2.266016,0.051377
3,DeepLearning_1_AutoML_20190419_001144,13.498404,3.674017,13.498404,2.463962,NaN
4,GBM_1_AutoML_20190419_015759,14.859340,3.854781,14.859340,2.370366,NaN
5,GBM_4_AutoML_20190419_001144,17.321442,4.161904,17.321442,2.420798,NaN
6,XGBoost_1_AutoML_20190418_233434,17.445320,4.176759,17.445320,2.447035,0.042466
7,XGBoost_1_AutoML_20190419_001144,18.109024,4.255470,18.109024,2.437325,0.044554
8,XGBoost_2_AutoML_20190418_233434,19.088113,4.368995,19.088113,2.489586,0.052634
9,XGBoost_3_AutoML_20190418_165441,19.616609,4.429064,19.616609,2.869058,0.079330


## Generating JSON file for all the models through FOR loop

In [19]:
import os.path
from pathlib import Path

In [20]:
aml_leaderboard_df=aml.leaderboard.as_data_frame()
model_set=aml_leaderboard_df['model_id']
mod_best=h2o.get_model(model_set[0])

In [21]:
##iterating over number of rows(all model_id), and generates the 'JSON' file with unique name each time, 
#if the particular file already exists then it will not overwrite it.

for i in range(model_set.shape[0]):
    mod_best = h2o.get_model(model_set[i])
    hy_parameter = mod_best.params
    n = run_id + str(i) +'_' + model_set[i] + '.json'
    file_name = Path(n)
    if not file_name.is_file():
        dict_to_json(hy_parameter, n)